In [17]:
# python提取照片坐标信息的代码如下所示：

# PS：Python小列子-读取照片位置

# Python exifread

# Python利用exifread库来解析照片的经纬度，对接百度地图API显示拍摄地点。
from PIL import Image
from PIL.ExifTags import TAGS
import os
output="result.csv"
out=open(output,'a')
out.write('lat,lon\n')
fpath="C:/Users/zero/Documents/BaiduNetdiskWorkspace"
for item in os.walk(fpath):
 ob=item[2]
 for i in ob:
  name=fpath+'/'+str(i)
  ret={}
  try:
   img=Image.open(name)
   if hasattr(img,'_getexif'):
    exifinfo=img._getexif()
    if exifinfo !=None:
     for tag,value in exifinfo.items():
      decoded=TAGS.get(tag,tag)
      ret[decoded]=value
      N1 = ret['GPSInfo'][2][0][0]
      N2 = ret['GPSInfo'][2][1][0]
      N3 = ret['GPSInfo'][2][2][0]
      N=int(N1)+int(N2)*(1.0/60)+int(N3)*(1.0/360000)
      E1 = ret['GPSInfo'][4][0][0]
      E2 = ret['GPSInfo'][4][1][0]
      E3 = ret['GPSInfo'][4][2][0]
      E=int(E1)+int(E2)*(1.0/60)+int(E3)*(1.0/360000)
      out.write(str(N)+','+str(E)+'\n')
  except:
   pass
out.close()






In [18]:
import exifread
import re
import json
import requests
def latitude_and_longitude_convert_to_decimal_system(*arg):
 """
 经纬度转为小数, 作者尝试适用于iphone6、ipad2以上的拍照的照片，
 :param arg:
 :return: 十进制小数
 """
 return float(arg[0]) + ((float(arg[1]) + (float(arg[2].split('/')[0]) / float(arg[2].split('/')[-1]) / 60)) / 60)
def find_GPS_image(pic_path):
 GPS = {}
 date = ''
 with open(pic_path, 'rb') as f:
  tags = exifread.process_file(f)
  for tag, value in tags.items():
   if re.match('GPS GPSLatitudeRef', tag):
    GPS['GPSLatitudeRef'] = str(value)
   elif re.match('GPS GPSLongitudeRef', tag):
    GPS['GPSLongitudeRef'] = str(value)
   elif re.match('GPS GPSAltitudeRef', tag):
    GPS['GPSAltitudeRef'] = str(value)
   elif re.match('GPS GPSLatitude', tag):
    try:
     match_result = re.match('\[(\w*),(\w*),(\w.*)/(\w.*)\]', str(value)).groups()
     GPS['GPSLatitude'] = int(match_result[0]), int(match_result[1]), int(match_result[2])
    except:
     deg, min, sec = [x.replace(' ', '') for x in str(value)[1:-1].split(',')]
     GPS['GPSLatitude'] = latitude_and_longitude_convert_to_decimal_system(deg, min, sec)
   elif re.match('GPS GPSLongitude', tag):
    try:
     match_result = re.match('\[(\w*),(\w*),(\w.*)/(\w.*)\]', str(value)).groups()
     GPS['GPSLongitude'] = int(match_result[0]), int(match_result[1]), int(match_result[2])
    except:
     deg, min, sec = [x.replace(' ', '') for x in str(value)[1:-1].split(',')]
     GPS['GPSLongitude'] = latitude_and_longitude_convert_to_decimal_system(deg, min, sec)
   elif re.match('GPS GPSAltitude', tag):
    GPS['GPSAltitude'] = str(value)
   elif re.match('.*Date.*', tag):
    date = str(value)
 return {'GPS_information': GPS, 'date_information': date}
def find_address_from_GPS(GPS):
 print(GPS)
 """
 使用Geocoding API把经纬度坐标转换为结构化地址。
 :param GPS:
 :return:
 """
 secret_key = 'xxxxxxxxxxxxxxxxxxxx'    # 百度地图创应用的秘钥 
 if not GPS['GPS_information']:
  return '该照片无GPS信息'
 lat, lng = GPS['GPS_information']['GPSLatitude'], GPS['GPS_information']['GPSLongitude']
 baidu_map_api = "http://api.map.baidu.com/geocoder/v2/?ak={0}&callback=renderReverse&location={1},{2}s&output=json&pois=0".format(
  secret_key, lat, lng)
 response = requests.get(baidu_map_api)
 content = response.text.replace("renderReverse&&renderReverse(", "")[:-1]
 baidu_map_address = json.loads(content)
 formatted_address = baidu_map_address["result"]["formatted_address"]
 # province = baidu_map_address["result"]["addressComponent"]["province"]
 # city = baidu_map_address["result"]["addressComponent"]["city"]
 # district = baidu_map_address["result"]["addressComponent"]["district"]
 return formatted_address
GPS_info = find_GPS_image(pic_path='lllll.jpg')  # 照片
address = find_address_from_GPS(GPS=GPS_info)
print(address)

ModuleNotFoundError: No module named 'exifread'

In [ ]:
def walkFile(file):
    for root, dirs, files in os.walk(file):

        # root 表示当前正在访问的文件夹路径
        # dirs 表示该文件夹下的子目录名list
        # files 表示该文件夹下的文件list

        # 遍历文件
        for f in files:
            print(os.path.join(root, f))

        # 遍历所有的文件夹
        for d in dirs:
            print(os.path.join(root, d))

In [2]:
import exifread
import re
import os
# output="result.csv"
# out=open(output,'a')
# out.write('lat,lon\n')
fpath="C:/Users/zero/Documents/BaiduNetdiskWorkspace"

'''# os.walk遍历文件名功能示例
for root, dirs, files in os.walk(fpath):
            # root 表示当前正在访问的文件夹路径
        # dirs 表示该文件夹下的子目录名list
        # files 表示该文件夹下的文件list

        # 遍历文件
        for f in files:
            print(os.path.join(root, f))

        # 遍历所有的文件夹
        for d in dirs:
            print(os.path.join(root, d))
'''

def walkFile(filepath):
    for root, dirs, files in os.walk(fpath):
        return files


fs=walkFile(fpath)
print(fs)


'''
def readExif(files):
    for f in files:
        fname=fpath+'/'+f
        try:
            f=open(fname)
            contents=exifread.process_file(f)
            for key in contents:
                if key == "GPS GPSLongitude":
                    print("经度: ", contents[key],contents['GPS GPSLatitudeRef'])
                    print("纬度: ",contents['GPS GPSLatitude'],contents['GPS GPSLongitudeRef'])
                    # print("高度基准: ",contents['GPS GPSAltitudeRef'])
                    # print("海拔高度: ",contents['GPS GPSAltitude'])
                # if re.match('Image Make', key):
                #     print('品牌信息: ' , contents[key])
                # if re.match('Image Model', key):
                #     print('具体型号: ' , contents[key])
                # if re.match('Image DateTime', key):
                #     print('拍摄时间: ' , contents[key])
                # # if re.match('EXIF ExifImageWidth', key):
                # #     print('照片尺寸: ' , contents[key],'*',contents['EXIF ExifImageLength'])
                # if re.match('Image ImageDescription',key):
                #     print('图像描述: ' , contents[key])

        except Exception as e:
            print('错误明细',e.__class__.__name__,e)
            continue

readExif(fs)

out.close()

# for f in files:
#     fname=fpath+'/'+f
#     print(fname)
#     ret={}
#     try:
#         img=Image.open(fname)
#         if hasattr(img,'_getexif'):
#             exifinfo=img._getexif()
#             if exifinfo !=None:
#                 for tag,value in exifinfo.items():
#                     decoded=TAGS.get(tag,tag)
#                     ret[decoded]=value
#                     N1 = ret['GPSInfo'][2][0][0]
#                     N2 = ret['GPSInfo'][2][1][0]
#                     N3 = ret['GPSInfo'][2][2][0]
#                     N=int(N1)+int(N2)*(1.0/60)+int(N3)*(1.0/360000)
#                     E1 = ret['GPSInfo'][4][0][0]
#                     E2 = ret['GPSInfo'][4][1][0]
#                     E3 = ret['GPSInfo'][4][2][0]
#                     E=int(E1)+int(E2)*(1.0/60)+int(E3)*(1.0/360000)
#                     out.write(str(N)+','+str(E)+'\n')
#     except Exception as e:
#         print('错误明细',e.__class__.__name__,e)
#         continue
# out.close()



#  ob=item[2]
#  for i in ob:
#   name=fpath+'/'+str(i)
#   ret={}
#   try:
#    img=Image.open(name)
#    if hasattr(img,'_getexif'):
#     exifinfo=img._getexif()
#     if exifinfo !=None:
#      for tag,value in exifinfo.items():
#       decoded=TAGS.get(tag,tag)
#       ret[decoded]=value
#       N1 = ret['GPSInfo'][2][0][0]
#       N2 = ret['GPSInfo'][2][1][0]
#       N3 = ret['GPSInfo'][2][2][0]
#       N=int(N1)+int(N2)*(1.0/60)+int(N3)*(1.0/360000)
#       E1 = ret['GPSInfo'][4][0][0]
#       E2 = ret['GPSInfo'][4][1][0]
#       E3 = ret['GPSInfo'][4][2][0]
#       E=int(E1)+int(E2)*(1.0/60)+int(E3)*(1.0/360000)
#       out.write(str(N)+','+str(E)+'\n')
#   except:
#    pass
# out.close()
'''

['1.jpg', '2.jpg', 'desktop.ini', 'IMG_20220219_114222.jpg', 'IMG_20220219_114237.jpg', 'IMG_20220219_114244.jpg', 'IMG_20220219_114337.jpg', 'IMG_20220219_114444.jpg', 'IMG_20220219_114652.jpg', 'IMG_20220219_114800.jpg', 'IMG_20220219_114839.jpg', 'IMG_20220219_115423.jpg', 'IMG_20220219_115429.jpg', 'IMG_20220219_115713.jpg', 'IMG_20220219_115917.jpg', 'IMG_20220219_120642.jpg', 'IMG_20220219_120705.jpg', 'IMG_20220219_121017.jpg', 'IMG_20220219_121025.jpg', 'IMG_20220219_121706.jpg', 'IMG_20220219_121745.jpg', 'IMG_20220219_122221.jpg', 'IMG_20220219_122324.jpg', 'IMG_20220219_123029.jpg', 'IMG_20220219_123908.jpg', 'IMG_20220219_131814.jpg', 'IMG_20220219_132108.jpg', 'IMG_20220219_132656.jpg', 'IMG_20220219_132734.jpg', 'IMG_20220219_133259.jpg', 'IMG_20220219_133506.jpg', 'IMG_20220219_133615.jpg', 'IMG_20220219_133955.jpg', 'IMG_20220219_135210.jpg', 'IMG_20220219_140635.jpg', 'IMG_20220219_140801.jpg', 'IMG_20220219_140925.jpg', 'IMG_20220219_141000.jpg', 'IMG_20220219_141001.

'\ndef readExif(files):\n    for f in files:\n        fname=fpath+\'/\'+f\n        try:\n            f=open(fname)\n            contents=exifread.process_file(f)\n            for key in contents:\n                if key == "GPS GPSLongitude":\n                    print("经度: ", contents[key],contents[\'GPS GPSLatitudeRef\'])\n                    print("纬度: ",contents[\'GPS GPSLatitude\'],contents[\'GPS GPSLongitudeRef\'])\n                    # print("高度基准: ",contents[\'GPS GPSAltitudeRef\'])\n                    # print("海拔高度: ",contents[\'GPS GPSAltitude\'])\n                # if re.match(\'Image Make\', key):\n                #     print(\'品牌信息: \' , contents[key])\n                # if re.match(\'Image Model\', key):\n                #     print(\'具体型号: \' , contents[key])\n                # if re.match(\'Image DateTime\', key):\n                #     print(\'拍摄时间: \' , contents[key])\n                # # if re.match(\'EXIF ExifImageWidth\', key):\n                # #     print(\'照片